## Machine Learning Applications for Health (COMP90089)
### Tutorial: Medical Imaging

> ### Goal: Pre-processing fundamentals steps for Images before training a model.


* Recap: What is a medical image? Any image that is **used for medical purposes**. 


This Tutorial uses de-identified CT image data, in DICOM format, provided by the Radiological Society of North America originally from a Kaggle competition [source](https://www.kaggle.com/c/rsna-intracranial-hemorrhage-detection/data) that attempt to predict Intracranial hemorrhage.


* What is a [DICOM file](https://www.dicomstandard.org/)? Digital Imaging and Communications in Medicine — is the international standard for medical images and related information.

###Read Tips:
* Article Review: "Recent advances and clinical applications of deep learning in medical image analysis" [read here.](https://www.sciencedirect.com/science/article/abs/pii/S1361841522000913)
* Comprehensive Tutorial for [preprocessing phase](https://www.kaggle.com/code/gzuidhof/full-preprocessing-tutorial/notebook)
* Tutorial Medical Image [Python](https://theaisummer.com/medical-image-python/)
* Understanding [CT windows, levels and densities](https://youtu.be/KZld-5W99cI)
* Pydicom python [Library](https://github.com/pydicom/pydicom)
* This Tutorial was based on this [source](https://towardsdatascience.com/medical-image-pre-processing-with-python-d07694852606)



---


> The first part of this Jupyter file is to mount the drive in Colab (acess files from your local drive) set up the main **libraries** and upload the necessary file.

* Mount the drive in your google colab

* Instal the pydicom library

In [ ]:
!pip install pydicom
!pip install opencv-python
!pip install scikit-image
!pip install requests

* Set up the main necessary libraries here

In [ ]:
import pydicom
import numpy
import numpy as np
import cv2
import os
import math
import pylab
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
from scipy import ndimage
from skimage import morphology
from io import BytesIO

### Reading DICOM images

We will be using the pydicom package to read the DICOM image using the `dcmread` function. However, assuming you are working on Google Colab, dealing with local files can be tedious, or at least requires extra effort to set up. Therefore, we are going to write a wrapper that uses an IO buffer to read the file directly from GitHub.

In [ ]:
def mydcmread(dicom_url):
    response = requests.get(dicom_url)
    response.raise_for_status()  # Check that the request was successful
    
    # Create a file-like buffer from the downloaded content
    dicom_buffer = BytesIO(response.content)
    
    # Use pydicom.dcmread() to read the DICOM file from the buffer
    dicom_data = pydicom.dcmread(dicom_buffer)
    return dicom_data
    

In [ ]:
file_path = 'https://github.com/melbourne-cdth/comp90089_medical_imaging_tutorial/raw/refs/heads/main/ID_000012eaf.dcm'


In [ ]:
#Access the file you just uploaded

medical_image = mydcmread(file_path)


* What is given by a dicom file? These files contain a lot of metadata (such as the pixel size, so how long one pixel is in every dimension in the real world). This pixel size of the scan differs from scan to scan.

In [ ]:
print(medical_image)

* How is the image shape?

In [ ]:
image = medical_image.pixel_array
print(image.shape)

* Check the minimum and maximum value

In [ ]:
print(image.min())
print(image.max())

* Some usefull functions while pre-processing image files. The unit of measurement in CT scans is the Hounsfield Unit (HU), which is a measure of radiodensity.

* In general the returned values are not in this unit. You should adjust them. HU unit by multiplying with the rescale slope and adding the intercept (which are conveniently stored in the metadata of the scans!).

* Some scanners have cylindrical scanning bounds, but the output image is square. The pixels that fall outside of these bounds get the fixed value -2000. The first step is setting these values to 0, which currently corresponds to air. 


In [ ]:
# The Hounsfield Unit (HU) is a relative quantitative measurement of the intensity of radio waves.
# More dense tissue, with greater X-ray beam absorption, has positive values and appears bright;
# Less dense tissue, with less X-ray beam absorption, has negative values and appears dark.
 
def transform_to_hu(medical_image, image):
    intercept = medical_image.RescaleIntercept
    slope = medical_image.RescaleSlope
    hu_image = image * slope + intercept

    return hu_image

## If you want a specific zone of the image you can adjust the windowing of image.
def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    
    return window_image





In [ ]:
def remove_noise(file_path, display=False):
    medical_image = mydcmread(file_path)
    image = medical_image.pixel_array
    
    hu_image = transform_to_hu(medical_image, image)
    brain_image = window_image(hu_image, 40, 80)
    
    segmentation = morphology.dilation(brain_image, np.ones((1, 1)))
    labels, label_nb = ndimage.label(segmentation)
    
    label_count = np.bincount(labels.ravel().astype(np.int64))
    label_count[0] = 0

    mask = labels == label_count.argmax()
    
    # Improve the brain mask
    mask = morphology.dilation(mask, np.ones((1, 1)))
    mask = ndimage.morphology.binary_fill_holes(mask)
    mask = morphology.dilation(mask, np.ones((3, 3)))
    
    masked_image = mask * brain_image

    if display:
        plt.figure(figsize=(15, 2.5))
        plt.subplot(141)
        plt.imshow(brain_image)
        plt.title('Original Image')
        plt.axis('off')
        
        plt.subplot(142)
        plt.imshow(mask)
        plt.title('Mask')
        plt.axis('off')

        plt.subplot(143)
        plt.imshow(masked_image)
        plt.title('Final Image')
        plt.axis('off')
    
    return masked_image

## Cropping image is needed to place the brain image at the center and get rid of unnecessary parts of image.

def crop_image(image, display=False):
    # Create a mask with the background pixels
    mask = image == 0

    # Find the brain area
    coords = np.array(np.nonzero(~mask))
    top_left = np.min(coords, axis=1)
    bottom_right = np.max(coords, axis=1)
    
    # Remove the background
    croped_image = image[top_left[0]:bottom_right[0],
                top_left[1]:bottom_right[1]]
    
    return croped_image

def add_pad(image, new_height=512, new_width=512):
    height, width = image.shape

    final_image = np.zeros((new_height, new_width))

    pad_left = int((new_width - width) // 2)
    pad_top = int((new_height - height) // 2)
    
    
    # Replace the pixels with the image's pixels
    final_image[pad_top:pad_top + height, pad_left:pad_left + width] = image
    
    return final_image

* Remove Noise function

In [ ]:
rnoise_2 = remove_noise(file_path, display=True)

* Tilt correction is the alignment of brain image in a proposed way. When tilt experienced by brain CT images, it may result in misalignment for medical applications. 

* It is important because when we train the model, it can see the whole data through the same alignment.

In [ ]:
img=numpy.uint8(rnoise_2)
contours, hier =cv2.findContours (img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
mask=numpy.zeros (img.shape, numpy.uint8)

# find the biggest contour (c) by the area
c = max(contours, key = cv2.contourArea)

(x,y),(MA,ma),angle = cv2.fitEllipse(c)

cv2.ellipse(img, ((x,y), (MA,ma), angle), color=(0, 255, 0), thickness=2)

rmajor = max(MA,ma)/2
if angle > 90:
    angle -= 90
else:
    angle += 90
xtop = x + math.cos(math.radians(angle))*rmajor
ytop = y + math.sin(math.radians(angle))*rmajor
xbot = x + math.cos(math.radians(angle+180))*rmajor
ybot = y + math.sin(math.radians(angle+180))*rmajor
cv2.line(img, (int(xtop),int(ytop)), (int(xbot),int(ybot)), (0, 255, 0), 3)

pylab.imshow(img)
pylab.show()

M = cv2.getRotationMatrix2D((x, y), angle-90, 1)  #transformation matrix

img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]), cv2.INTER_CUBIC)

pylab.imshow(img)
pylab.show()

* Cropped Image

In [ ]:
croppedImage = crop_image(img)
plt.figure(figsize=(15, 5))
plt.imshow(croppedImage)

In [ ]:
croppedImage.shape

### Final Image with applied preprocessing steps: cleaned and centered.

In [ ]:
plt.figure(figsize=(15, 25))
final_image = add_pad(croppedImage)
plt.imshow(final_image)
